In [ ]:
## Extraer frames de los vídeos

import cv2
import os
from tqdm import tqdm

# === PARÁMETROS ===
videos_dir = "videos"              # Carpeta con los vídeos
output_dir = "frames_extraidos"    # Carpeta para guardar resultados
sampling_interval_ms = 300         # Intervalo de muestreo en ms
diff_threshold = 25                # Umbral de diferencia para guardar

video_extensions = ('.mp4', '.avi', '.mov', '.mkv')

# === Crear directorio de salida general ===
os.makedirs(output_dir, exist_ok=True)

# === Listar vídeos ===
video_files = [f for f in os.listdir(videos_dir) if f.lower().endswith(video_extensions)]
if not video_files:
    print("No se encontraron vídeos en la carpeta.")
    exit()

print(f"Vídeos encontrados: {len(video_files)}")
for v in video_files:
    print(f"   - {v}")

# === Calcular total estimado de frames para barra global ===
print("\nEscaneando duración de todos los vídeos...")
total_frames_estimate = 0
for video_file in video_files:
    cap = cv2.VideoCapture(os.path.join(videos_dir, video_file))
    total = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    total_frames_estimate += max(0, total)
    cap.release()

print(f"Frames estimados totales a procesar (sin skips): ~{int(total_frames_estimate)}\n")

# === Procesamiento ===
global_progress = tqdm(total=int(total_frames_estimate), desc="Progreso global", unit="frame")

total_saved = 0

for video_file in video_files:
    print(f"\nProcesando vídeo: {video_file}")
    video_path = os.path.join(videos_dir, video_file)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error al abrir: {video_file}")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 25

    frame_interval = int((sampling_interval_ms / 1000) * fps)

    print(f"   -> FPS: {fps:.2f}")
    print(f"   -> Saltando cada {frame_interval} frames aproximadamente.")

    # Subcarpeta para este vídeo
    video_output_subdir = os.path.join(output_dir, os.path.splitext(video_file)[0])
    os.makedirs(video_output_subdir, exist_ok=True)

    prev_gray = None
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Barra de progreso global
        global_progress.update(1)

        if frame_count % frame_interval != 0:
            frame_count += 1
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is None:
            save_path = os.path.join(video_output_subdir, f"frame_{frame_count:06d}.jpg")
            cv2.imwrite(save_path, frame)
            saved_count += 1
            prev_gray = gray
            frame_count += 1
            continue

        diff = cv2.absdiff(gray, prev_gray)
        mean_diff = diff.mean()

        if mean_diff > diff_threshold:
            save_path = os.path.join(video_output_subdir, f"frame_{frame_count:06d}.jpg")
            cv2.imwrite(save_path, frame)
            saved_count += 1
            prev_gray = gray

        frame_count += 1

    cap.release()
    print(f"Frames guardados de {video_file}: {saved_count}")
    total_saved += saved_count

global_progress.close()
print(f"\nExtracción completa para todos los vídeos. Total de frames guardados: {total_saved}")


Vídeos encontrados: 31
   - 00000000742000000.mp4
   - 00000000742000100.mp4
   - 00000000764002600.mp4
   - 00000000764002700.mp4
   - 00000000764002800.mp4
   - 00000000764003000.mp4
   - 00000000764003100.mp4
   - 00000000764003200.mp4
   - 00000000764003300.mp4
   - 00000000787000000.mp4
   - 00000002740000000.mp4
   - 00010002417000000.mp4
   - 00010002427000200.mp4
   - 00010003234000200.mp4
   - 0100000025100000000.mp4
   - 0100000025500010000.mp4
   - 0100000029500000000.mp4
   - 01010000237000000.mp4
   - 01010000343000000.mp4
   - 01010000347000000.mp4
   - 10.61.88.103_35_D20240521170239_20240521170440.mp4
   - 10.61.88.103_35_D20240521170538_20240521170812.mp4
   - 10.61.88.103_35_D20240521170847_20240521171122.mp4
   - 10.61.88.103_35_D20240521171154_20240521171556.mp4
   - Augusto 1_P10fsx_20221120020000_20221120062959.mp4
   - Augusto 1_P5-3fsx_20221120020000_20221120052208.mp4
   - ch01_00000000014000113.mp4
   - Dulcinea O&M_CF07_20250117060001_20250117090001.mp4
   - 

Progreso global:   0%|          | 14/6342760 [00:00<12:45:14, 138.14frame/s]


Procesando vídeo: 00000000742000000.mp4
   -> FPS: 25.00
   -> Saltando cada 7 frames aproximadamente.


Progreso global:   1%|          | 47595/6342760 [02:00<5:05:39, 343.25frame/s]

Frames guardados de 00000000742000000.mp4: 2

Procesando vídeo: 00000000742000100.mp4
   -> FPS: 25.00
   -> Saltando cada 7 frames aproximadamente.


...

Progreso global:  10%|█         | 664793/6342760 [23:29<1:01:06, 1548.42frame/s]

Frames guardados de 00010002427000200.mp4: 1

Procesando vídeo: 00010003234000200.mp4
   -> FPS: 25.00
   -> Saltando cada 7 frames aproximadamente.


Progreso global:  11%|█         | 697415/6342760 [23:36<19:16, 4880.81frame/s]  

Frames guardados de 00010003234000200.mp4: 1

Procesando vídeo: 0100000025100000000.mp4
   -> FPS: 20.00
   -> Saltando cada 6 frames aproximadamente.


Progreso global:  11%|█         | 698683/6342760 [23:46<8:34:35, 182.80frame/s]

In [ ]:
## Juntar todos los frames en una misma carpeta

import os
import cv2

input_root = "frames_extraidos" # Carpeta raíz con subcarpetas de frames
output_dir = "frames_unicos"    # Carpeta destino
os.makedirs(output_dir, exist_ok=True)

counter = 1

for subfolder in sorted(os.listdir(input_root)):
    subfolder_path = os.path.join(input_root, subfolder)
    if not os.path.isdir(subfolder_path):
        continue

    print(f"Procesando subcarpeta: {subfolder}")

    for filename in sorted(os.listdir(subfolder_path)):
        if filename.lower().endswith((".jpg", ".png")):
            src_path = os.path.join(subfolder_path, filename)

            # Cargar imagen con OpenCV
            img = cv2.imread(src_path)
            if img is None:
                print(f"No se pudo leer {src_path}")
                continue

            # Nuevo nombre
            new_filename = f"vigilancia_{counter:06d}.jpg"
            dst_path = os.path.join(output_dir, new_filename)

            # Guardar como JPG
            cv2.imwrite(dst_path, img)

            counter += 1

print(f"Proceso completado. Total de imágenes copiadas: {counter-1}")


Procesando subcarpeta: 00000000742000000
Procesando subcarpeta: 00000000742000100
Procesando subcarpeta: 00000000764002600
Procesando subcarpeta: 00000000764002700
Procesando subcarpeta: 00000000764002800
Procesando subcarpeta: 00000000764003000
Procesando subcarpeta: 00000000764003100
Procesando subcarpeta: 00000000764003200
Procesando subcarpeta: 00000000764003300
Procesando subcarpeta: 00000000787000000
Procesando subcarpeta: 00000002740000000
Procesando subcarpeta: 00010002417000000
Procesando subcarpeta: 00010002427000200
Procesando subcarpeta: 00010003234000200
Procesando subcarpeta: 0100000025100000000
Proceso completado. Total de imágenes copiadas: 283
